# Improved imreg helper

*Based on my current code I would like to test an improved imreg helper.* 

With: 
* cleaner code, 
* 2x4 corner maps, 
* input of existing corner coordinates 

In [ ]:
%load_ext autoreload
%autoreload 2 
%matplotlib notebook

## Loading source and destiny images 

In [ ]:
import os 
os.chdir('/home/frank/Work/Projecten/DoRe/viz/raw_nmf')

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import skimage.exposure as ske

from maxrf4u import DataStack

In [ ]:
ds = DataStack('RP-T-1898-A-3689.datastack') 

x_keVs = ds.read('maxrf_energies')
y_max = ds.read('maxrf_maxspectrum') 
cube = ds.read('maxrf_cube', compute=False) # don't load into memory yet (too big)

is_iron_Ka_band = (x_keVs > 6.1) * (x_keVs < 6.7) 
np.sum(is_iron_Ka_band)

60

In [ ]:
FeKa_slice = cube[:,:,is_iron_Ka_band].compute() # load only this slice into memory 
FeKa_map = FeKa_slice.sum(axis=2) / 60 # number of channels in the Fe_Ka band 

In [ ]:
susanna_highres_file = '/home/frank/Work/Projecten/DoRe/data/drawings/rma-web-highres/RP-T-1898-A-3689_highres.png'
susanna_highres = plt.imread(susanna_highres_file)

im_src = susanna_highres
im_dst = np.clip(FeKa_map, a_min=0, a_max=2) 

# Ipyleaflet 

https://towardsdatascience.com/ipyleaflet-interactive-mapping-in-jupyter-notebook-994f19611e79

    icon_url = “http://icons.iconarchive.com/icons/pelfusion/long-shadow-media/512/Maps-Pin-Place-icon.png"
    
    icon = Icon(icon_url=icon_url)
    
    mark = Marker(location=[57.719503, 12.008843], icon=icon, rotation_angle=0, rotation_origin=’128px 128px’)

SVG as icons: https://www.python-graph-gallery.com/312-add-markers-on-folium-map

    folium.Marker(
            location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
            popup=popup,
            icon=folium.DivIcon(html=f"""
                <div><svg>
                    <circle cx="50" cy="50" r="40" fill="#69b3a2" opacity=".4"/>
                    <rect x="35", y="35" width="30" height="30", fill="red", opacity=".3" 
                </svg></div>""")

In [ ]:
from ipyleaflet import (Map, ImageOverlay, FullScreenControl, projections, Marker, Rectangle, Icon, AwesomeIcon,
                        WidgetControl, DivIcon, Popup) 
from ipywidgets import Layout, VBox, GridBox, Textarea, HTML, jslink, FloatText 
from ipywidgets.embed import embed_minimal_html 

from maxrf4u import img_to_url 

h_src, w_src = im_src.shape[0:2]
h_dst, w_dst = im_dst.shape[0:2]

url_src, shape_src = img_to_url(im_src, max_width=500)
url_dst, shape_dst = img_to_url(im_dst/im_dst.max())

In [ ]:
def extract_marker_points(markers, im_src, im_dst, roundoff=True): 
    '''Extracts current marker locations from `markers` and flips into image coordinates. 
    
    Returns: pts_src, pts_dst '''
    
    h_src = im_src.shape[0]
    h_dst = im_dst.shape[0]
    
    markers_src = markers[0:4] 
    markers_dst = markers[4:8]
    
    pts_src = []
    for mrk in markers_src: 
        
        y_flip, x = mrk.location
        xy = x, h_src - y_flip 
        pts_src.append(xy)
        
        
    pts_src = np.array(pts_src)
    if roundoff: 
        pts_src = np.round(pts_src, decimals=2)
        
    pts_dst = []
    for mrk in markers_dst: 
        
        y_flip, x = mrk.location
        xy = x, h_dst - y_flip 
        pts_dst.append(xy)
        
    pts_dst = np.array(pts_dst)
    
    if roundoff: 
        pts_dst = np.round(pts_dst, decimals=2)
        
    return pts_src, pts_dst 

In [ ]:
layout = Layout(width='90%', height='200px')
grid_layout = Layout(grid_template_columns='repeat(4, 25%)') 
crs = projections['Simple']
fsc = FullScreenControl() 

# default marker locations  
mloc_src = [[h_src, 0], [h_src, w_src], [0, 0], [0, w_src]]
mloc_dst = [[h_dst, 0], [h_dst, w_dst], [0, 0], [0, w_dst]]

# marker colors 
colors = ['red', 'green', 'blue', 'orange']

# svg 

#icon=DivIcon(html=f"""
#                  <div><svg>
#                  <circle cx="50" cy="50" r="40" fill="#69b3a2" opacity=".4"/>
#                  <rect x="35", y="35" width="30" height="30", fill="red", opacity=".3" 
#                  </svg></div>""")

icon_url = 'http://icons.iconarchive.com/icons/pelfusion/long-shadow-media/512/Maps-Pin-Place-icon.png'



icon = Icon(icon_url=icon_url, icon_size=[50, 50])

#create map widgets 
maps = []
markers = []

# source image maps 

for location, color in zip(mloc_src, colors): 
    
    m = Map(center=[0.5*h_src, 0.5*w_src], zoom=-5, crs=crs, layout=layout, 
            scroll_wheel_zoom=True, min_zoom=-10, interpolation='nearest') 
       
    imo_src = ImageOverlay(url=url_src, bounds=[[-0.5, -0.5], [h_src+0.5, w_src+0.5]]) 
    #icon = AwesomeIcon(name='circle-o', marker_color=color)
    
    #mrk = Marker(location=location, icon=icon)
    mrk = Marker(location=location, icon=icon)
    markers.append(mrk)
    
    m.add(fsc) 
    m.add(imo_src)
    m.add(mrk)
    m.remove(m.layers[0]) # hack to remove world map 
    
    maps.append(m)

# destination image maps 

for location, color in zip(mloc_dst, colors): 
    
    m = Map(center=[0.5*h_dst, 0.5*w_dst], zoom=-5, crs=crs, layout=layout, 
            scroll_wheel_zoom=True, min_zoom=-10, interpolation='nearest', attribution='xyz')
    
    imo_dst = ImageOverlay(attribution='abc', url=url_dst, bounds=[[-0.5, -0.5], [h_dst+0.5, w_dst+0.5]]) 
    
    icon = AwesomeIcon(name='circle-o', marker_color=color)
    mrk = Marker(location=location, icon=icon, title='xyz') 
    markers.append(mrk)
    
    m.add(fsc) 
    m.add(imo_dst)
    m.add(mrk)
    m.remove(m.layers[0]) # hack to remove world map 
    
    maps.append(m)    

gridbox = GridBox(maps, layout=grid_layout)

gridbox

GridBox(children=(Map(center=[2460.0, 2198.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

In [ ]:
import ipyleaflet as leaf

In [ ]:
leaf.DivIcon??

In [ ]:
import os
os.getcwd()

'/mnt/datadisk/Work/Projecten/DoRe/viz/raw_nmf'

In [ ]:
AwesomeIcon??

In [ ]:
embed_minimal_html('gridbox.html', gridbox)

In [ ]:
html = '<div style="color: red; background-color: rgba(0, 255, 0, 0.5)">text</div>'

div_icon = DivIcon(html=html, bg_pos=location, icon_anchor=[0, 0], icon_size=[20, 20])

In [ ]:
textbox.add_traits()

<bound method HasTraits.trait_values of DivIcon(bg_pos=[0, 1592], html='<div style="color: red; background-color: rgba(0, 255, 0, 0.5)">text</div>', icon_anchor=[0, 0], icon_size=[20, 20], options=['bg_pos', 'html', 'icon_anchor', 'icon_size', 'popup_anchor'], popup_anchor=[0, 0])>

In [ ]:
extract_marker_points(markers, im_src, im_dst)

(array([[-224.1, -232. ],
        [4396. ,    0. ],
        [   0. , 4920. ],
        [4396. , 4920. ]]),
 array([[ 160.07,  156.  ],
        [1592.  ,    0.  ],
        [   0.  , 1692.  ],
        [1592.  , 1692.  ]]))

# Gridbox code 

In [ ]:
mr

[4919.953125, -0.015625]

In [ ]:
import json 
import os 

import IPython 

from IPython.display import display

from ipywidgets.embed import embed_minimal_html, embed_data 
from ipywidgets import Layout, HBox, VBox, GridBox, jslink, HTML 
from ipywidgets import Widget 

from ipyleaflet import (Map, projections, ImageOverlay, Rectangle, ZoomControl, FullScreenControl, 
                        DrawControl, WKTLayer, Popup)

from maxrf4u import img_to_url 


def make_gridbox_widget(images, titles): 
    
    # close all widgets to avoid ever growing html export file size
    print('Closing all widgets...')
    Widget.close_all()
     
    # convert numpy arrays to color compressed png/base64 url strings to be used in html output    
    print('Compressing images...') 
    shapes = [im.shape for im in images]
    urls = [img_to_url(im)[0] for im in images]
    lengths = [len(u) for u in urls]
    size = sum(lengths)
    
    print(f'Ready! Total string length: {size / 1e6:.02f} MB')
     
    layout = Layout(width='300px', height='300px')
    grid_layout = Layout(grid_template_columns="repeat(5, 305px)") 
    
    # make map widgets  
    
    map_widgets = [] 
    for url in urls:

        m = Map(center=[150, 150], zoom=0, crs=projections['Simple'], layout=layout, 
                    scroll_wheel_zoom=True, min_zoom=-10)#, interpolation='nearest')
        imo = ImageOverlay(url=url, bounds=[[0, 0], [300, 300]])
        fsc = FullScreenControl()
        
        m.add(imo)
        m.add(fsc)
        m.remove(m.layers[0]) # hack to remove world map 
        
        map_widgets.append(m)
        
    # sync map widgets centers and zoom levels via browser javascript 
    first, rest = map_widgets[0], map_widgets[1:]
    for r in rest: 
        jslink([first, 'center'],[r, 'center'])
        jslink([first, 'zoom'],[r, 'zoom'])
    
    # titles 
    
    title_widgets = []
    for t in titles: 
        
        tw = HTML(f'<center><bf>{t}</bf></center>') # ? 
        title_widgets.append(tw)
    
    # combine titles and maps 
    vboxes = []
    for title_widget, map_widget in zip(title_widgets, map_widgets): 
        
        vbox = VBox([title_widget, map_widget], layout=layout)
        vboxes.append(vbox)
    
    gridbox = GridBox(vboxes, layout=grid_layout)
    
    return gridbox 

In [ ]:
from IPython.display import display 
import ipyleaflet as ipl 
from ipywidgets import Text, HTML, Layout, HBox, VBox 
import ipywidgets 
from ipyleaflet import Map, ImageOverlay, basemap_to_tiles, Marker, WidgetControl
  
import io 
import matplotlib.pyplot as plt
import numpy as np
import base64 
import skimage.transform as skt 
#from skimage.io import imread
import imageio 

import skimage 

import numpy as np
import cv2 


def img_to_url(img_data, max_width=None): 
    '''Filename or numpy array *img_data* is transformed into 256 bit color compressed base64 encoded url string. 
    
    To reduce image size specify thumbnail `max_width`. 
    
    Returns: url_string, shape'''
        
    try: 
        # better than matplotlib use skimage.io.imread to avoid float explosion...
        img = imageio.imread(img_data) # if img_data is an image file path 
    except: 
        img = img_data # otherwise assume img_data is an image like numpy array  
        
    shape = img.shape[0:2] # height and width only 
    h, w = shape 
    
    # rescaling image if width > max_width 
    if max_width is not None:  
        if w > max_width:
            scale = max_width / w  
            img = skt.rescale(img, scale, multichannel=True) 
            
    # reduce colors to 256 levels to keep file size minimal 
    img_ubyte = skimage.util.img_as_ubyte(img)
    
    # write to buffer 
    buff = io.BytesIO();
    plt.imsave(buff, img_ubyte, format='png')
   
    
    # convert to base64 string 
    base64_string = base64.b64encode(buff.getvalue()).decode("ascii")
    url_string = f'data:image/png;base64,{base64_string}' 
    
    # let's close buffer just in case
    buff.close()
    
 
    return url_string, shape 


def mark_corners(map_, img_shape): 

    h, w = img_shape[0:2] 

    corners = [[h, 0], [h, w], [0, w], [0, 0]] 
    colors = ['red', 'green', 'blue', 'orange']
    labels = ['top left', 'top right', 'bottom right', 'bottom left']

    markers = [] 

    for location, color, label in zip(corners, colors, labels): 
        icon = ipl.AwesomeIcon(name='circle-o', marker_color=color)
        marker = Marker(location=location, draggable=True, title=f'Drag me to {label} landmark', icon=icon)
        markers.append(marker)
        map_.add(marker) 
    
    return markers 


# callback 
def on_marker_move(change): 
    
    textbox.value = extract_corner_points(src_markers, src_shape, dst_markers, dst_shape, return_text=True)

    
    
def extract_corner_points(src_markers, src_shape, dst_markers, dst_shape, return_text=False): 
    '''Flip coordinates to make top of image zero. 
    
    Returns: pts_string 
    '''
    
    src_pts = []
    h, w = src_shape 
    for marker in src_markers:   
        y, x = marker.location 
        xy_flip = [ x - 0.5, h - y - 0.5]
        src_pts.append(xy_flip)
             
    dst_pts = []
    h, w = dst_shape  
    for marker in dst_markers:   
        y, x = marker.location 
        xy_flip = [ x - 0.5, h - y - 0.5]
        dst_pts.append(xy_flip)
        
    if return_text:     
        # two decimal strings for textbox 
        src_pts_string = 'src_pts = ' + ', '.join([f'[{y:.02f}, {x:.02f}]' for [y, x] in src_pts])
        dst_pts_string = 'dst_pts = ' + ', '.join([f'[{y:.02f}, {x:.02f}]' for [y, x] in dst_pts])

        pts_string = f'# corner points for image registration: \n{src_pts_string}\n{dst_pts_string}'

        return pts_string 
    
    else: 
        return src_pts, dst_pts 

    
    
class ImageRegistrationHelper(object): 
    
    def on_marker_move_(self, change): 
            
            self.textbox.value = extract_corner_points(self.src_markers, self.src_shape, 
                                                       self.dst_markers, self.dst_shape, 
                                                       return_text=True) 
            
    def get_corner_points_for_image_registration(self, verbose=True): 
        '''Get 4 source and 4 destination corner points for image registration. 
        
        Extracts current positions from all markers. Output can be used in warp function. 
        
        Returns: src_points, dst_points
        '''
        
        src_points, dst_points = extract_corner_points(self.src_markers, self.src_shape, 
                                           self.dst_markers, self.dst_shape, 
                                           return_text=False)
        
        print(f'src_points = {src_points}\ndst_points = {dst_points}')
        
        src_points = np.array(src_points) 
        dst_points = np.array(dst_points)
        
        return src_points, dst_points 
        
    
    def __init__(self, src_imdata, dst_imdata): 
        
        layout = Layout(width='500px', height='500px')
        resolutions = [256 * 2 ** (-l) for l in range(20)]
 
        crs = dict(name='Pixelcoords', custom=True, bounds=[[0, 0], [1000, 1000]], origin=[0, 0], 
                    resolutions=resolutions, proj4def='+proj=longlat')
    

        # read images and convert to png/base64 url  
        src_url, self.src_shape = img_to_url(src_imdata) 
        dst_url, self.dst_shape = img_to_url(dst_imdata) 
        
        
        src_h, src_w = self.src_shape 
        self.src_map = Map(scroll_wheel_zoom=True, crs=crs, center=[src_h/2, src_w/2], zoom=4, 
                      layout=layout, interpolation='nearest')

        dst_h, dst_w = self.dst_shape
        self.dst_map = Map(scroll_wheel_zoom=True, crs=crs, center=[dst_h/2, dst_w/2], zoom=4, 
                      layout=layout, interpolation='nearest')
        
        self.src_markers = mark_corners(self.src_map, self.src_shape)
        self.dst_markers = mark_corners(self.dst_map, self.dst_shape)

        src_imo = ImageOverlay(url=src_url, bounds=[[0, 0], self.src_shape]) # SW and NE corners
        dst_imo = ImageOverlay(url=dst_url, bounds=[[0, 0], self.dst_shape]) # SW and NE corners

        self.src_map.add(src_imo)
        self.dst_map.add(dst_imo)

        # this hack removes the default world map layer 
        # (seems I can do that only after creating other widgets) 
        self.src_map.remove(self.src_map.layers[0]) 
        self.dst_map.remove(self.dst_map.layers[0])

        for marker in self.src_markers: 
            marker.observe(self.on_marker_move_)

        for marker in self.dst_markers: 
            marker.observe(self.on_marker_move_)

        # initialize textbox    
        self.pts_string = extract_corner_points(self.src_markers, self.src_shape, 
                                           self.dst_markers, self.dst_shape, 
                                           return_text=True)

        self.textbox = ipywidgets.Textarea(self.pts_string, rows=3, layout=Layout(width='100%'))

        self.combi = VBox([HBox([self.src_map, self.dst_map]), self.textbox])
        
        display(self.combi)



        
def warp(im_src, im_dst, pts_src, pts_dst, keep_scale=True, rgba=True, alpha_color=[1, 0, 0]): 
    '''Opencv based homographic registration. Can return transparent overlay (rgba). 
    
    Returns: *im_warped*, *extent* '''

    src_h, src_w = im_src.shape[0:2]
    dst_h, dst_w = im_dst.shape[0:2]
    
    if keep_scale: 
        scale = src_h / dst_h 
    else: 
        scale = 1

    # calculate homography 
    hom, status = cv2.findHomography(pts_src, scale * pts_dst)

    # Size is nearest integer scaled (width,height) of im_dst
    size = round(scale * dst_w), round(scale * dst_h)
    
    # warp im_src onto destination 
    im_warped = cv2.warpPerspective(im_src, hom, size) 
    
    # warp white onto destination to create mask 
    mask_src = np.ones([src_h, src_w])
    mask_warped = cv2.warpPerspective(mask_src, hom, size) 
    
    # calculate extent for plotting 
    h, w = im_dst.shape[0:2] 
    extent =  (0, w, h, 0) 
    
    # convert into rgba image  
    if rgba is True: 
        im_rgba = np.ones([h, w, 4])

        # for single channel image 
        if len(im_src.shape) == 2:
            # colorize
            im_rgba[:,:,0:3] = alpha_color[0:3] 
            #  make alpha layer 
            im_rgba[:,:,3] = im_warped 
            
        # otherwise assume image is rgb
        elif len(im_warped.shape) == 3: 
            
            # rgb channels
            im_rgba[:,:,0:3] = im_warped[:,:,0:3]
            # alpha channels 
            im_rgba[:,:,3] = mask_warped
       
        im_warped = im_rgba 
    
    return im_warped, extent  

